In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')


# Intro to Bayesian Inference


In [ ]:
import torch
import pyro
import pyro.distributions as dist
from matplotlib.ticker import StrMethodFormatter
from pyro.infer import MCMC, NUTS, Predictive
import arviz as az
import matplotlib.pyplot as plt

from gempy_probability.plot_posterior import PlotPosterior

y_obs = torch.tensor([2.12])
y_obs_list = torch.tensor([2.12, 2.06, 2.08, 2.05, 2.08, 2.09,
                           2.19, 2.07, 2.16, 2.11, 2.13, 1.92])
pyro.set_rng_seed(4003)

In [ ]:
def model(conf, data):
    if conf in ['n_s', 'n_o']:
        mu = pyro.sample('$\mu$', dist.Normal(2.08, 0.07))
    elif conf in ['u_s', 'u_o']:
        mu = pyro.sample('$\mu$', dist.Uniform(0, 10))
    sigma = pyro.sample('$\sigma$', dist.Gamma(0.3, 3))
    y = pyro.sample('$y$', dist.Normal(mu, sigma), obs=data)
    return y


def infer_model(config, data):
    # 1. Prior Sampling
    prior = Predictive(model, num_samples=100)(config, data)
    # 2. MCMC Sampling
    nuts_kernel = NUTS(model)
    mcmc = MCMC(nuts_kernel, num_samples=100, warmup_steps=100)  # Assuming 1000 warmup steps
    mcmc.run(config, data)
    # Get posterior samples
    posterior_samples = mcmc.get_samples()
    # 3. Sample from Posterior Predictive
    posterior_predictive = Predictive(model, posterior_samples)(config, data)
    # %%
    az_data = az.from_pyro(
        posterior=mcmc,
        prior=prior,
        posterior_predictive=posterior_predictive
    )

    return az_data

The *data generating process* is latent. Therefore, it will be impossible to perfectly describe it. At this point,
we need to compromise and start to make assumptions and choose a model.
Any probabilistic family, \( \pi(y;\varTheta) \), consists of two distinct sets: observations, \( y \) and model
parameters \( \varTheta \). Depending on which set is fixed, we obtain either (i) the forward view: for a given set 
of \( \varTheta \) there is a probability of sample \( y \) or (ii) the inverse view: the observed \( y \) is so much 
likely for these values of \( \varTheta \). Discussing the nuances between assuming that the model emerge from 
observations or if the observation are generated by the model are beyond the scope of this paper. Here, we stand on 
an eclectic position, arguing that these options are not mutually excluding, any (prior) model that the brain can 
come up are substantiated—in one way or another—on direct observations of the present or in observations of the past 
consolidated in form of experience. This reconciliation of statistical approaches it is formalised as Empirical Bayes. 
In chapter XX, we will deepen on this topic while describing the proposed network for structural geology, for now, 
let’s assume that we perform inverse statistics and hence we fix the observations \( y \). For this example we can 
assume that the generating process belongs to the Gaussian family:



TODO: Make the graph of the model



In [ ]:
az_data = infer_model("u_s", y_obs_list)
az.plot_trace(az_data)
plt.show()

In [ ]:
p = PlotPosterior(az_data)
p.create_figure(figsize=(9, 3), joyplot=False, marginal=False)
p.plot_normal_likelihood('$\mu$', '$\sigma$', '$y$', iteration=-1, hide_bell=True)
p.likelihood_axes.set_xlim(1.90, 2.2)
p.likelihood_axes.xaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
for tick in p.likelihood_axes.get_xticklabels():
    tick.set_rotation(45)
plt.show()

No matter which probability density function we choose, for real applications we will never find the exact data 
generated process---neither we will be able to say if we have found it for that matter---due to an oversimplification 
of reality. For most applications, the usual families of probability density functions and transformations of those 
are more than enough approximations for the purpose of the model. In Chapter [sec:model_selection], we will delve into 
this topic.

Once the model is defined we need to infer the set of parameters \( \varTheta \) of the family of density functions 
over the observational space, \( \pi_S(y;\varTheta) \). In the case of the normal family, we need to infer the value 
of the mean, \( \mu \) and standard deviation \( \sigma \). Up to this point, all the description of the probabilistic 
modelling is agnostic in relation to Frequentist or Bayesian views. These two methodologies diverge on how they 
infer \( \varTheta \).



In [ ]:
p = PlotPosterior(az_data)
p.create_figure(figsize=(9, 3), joyplot=False, marginal=False)
p.plot_normal_likelihood('$\mu$', '$\sigma$', '$y$', iteration=-1, hide_lines=True)
p.likelihood_axes.set_xlim(1.70, 2.40)
p.likelihood_axes.xaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
for tick in p.likelihood_axes.get_xticklabels():
    tick.set_rotation(45)
plt.show()

In [ ]:
p = PlotPosterior(az_data)

p.create_figure(figsize=(9, 9), joyplot=True, marginal=False, likelihood=False, n_samples=31)
p.plot_joy(('$\mu$', '$\sigma$'), '$y$', iteration=14)
plt.show()

In [ ]:
p = PlotPosterior(az_data)

p.create_figure(figsize=(9, 5), joyplot=False, marginal=True, likelihood=True)
p.plot_marginal(var_names=['$\mu$', '$\sigma$'],
                plot_trace=False, credible_interval=.93, kind='kde',
                joint_kwargs={'contour': True, 'pcolormesh_kwargs': {}},
                joint_kwargs_prior={'contour': False, 'pcolormesh_kwargs': {}})

p.plot_normal_likelihood('$\mu$', '$\sigma$', '$y$', iteration=-1, hide_lines=True)
p.likelihood_axes.set_xlim(1.70, 2.40)
plt.show()

In [ ]:
az_data = infer_model("u_o", y_obs)

In [ ]:
p = PlotPosterior(az_data)

p.create_figure(figsize=(9, 5), joyplot=False, marginal=True, likelihood=True)
p.plot_marginal(var_names=['$\mu$', '$\sigma$'],
                plot_trace=False, credible_interval=.93, kind='kde',
                joint_kwargs={'contour': True, 'pcolormesh_kwargs': {}},
                joint_kwargs_prior={'contour': False, 'pcolormesh_kwargs': {}})

p.axjoin.set_xlim(1.96, 2.22)
p.plot_normal_likelihood('$\mu$', '$\sigma$', '$y$', iteration=-6, hide_lines=True)
p.likelihood_axes.set_xlim(1.70, 2.40)
plt.show()

In [ ]:
az_data = infer_model("u_s", y_obs_list)
p = PlotPosterior(az_data)

p.create_figure(figsize=(9, 5), joyplot=False, marginal=True, likelihood=True)
p.plot_marginal(var_names=['$\mu$', '$\sigma$'],
                plot_trace=False, credible_interval=.93, kind='kde',
                joint_kwargs={'contour': True, 'pcolormesh_kwargs': {}},
                joint_kwargs_prior={'contour': False, 'pcolormesh_kwargs': {}})

p.plot_normal_likelihood('$\mu$', '$\sigma$', '$y$', iteration=-5, hide_lines=True)
p.axjoin.set_xlim(1.96, 2.22)
p.likelihood_axes.set_xlim(1.70, 2.4)
plt.show()

# License
The code in this case study is copyrighted by Miguel de la Varga and licensed under the new BSD (3-clause) license:

https://opensource.org/licenses/BSD-3-Clause

The text and figures in this case study are copyrighted by Miguel de la Varga and licensed under the CC BY-NC 4.0 license:

https://creativecommons.org/licenses/by-nc/4.0/
Make sure to replace the links with actual hyperlinks if you're using a platform that supports it (e.g., Markdown or HTML). Otherwise, the plain URLs work fine for plain text.

